In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
!pip -q install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.5/35.5 MB 58.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 131.4 MB/s eta 0:00:00


In [ ]:
!pip install datasets

In [ ]:
!pip install accelerate

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import evaluate
from transformers import (
    BartForConditionalGeneration,
    PreTrainedTokenizerFast,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
import re
from kiwipiepy import Kiwi
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast, TrainingArguments, Trainer
import torch
from transformers import BartTokenizer
import seaborn as sns

In [ ]:
data = pd.read_csv('./제주도 프로젝트/final_preprocessed_data.csv')
data

,dialect_form,standard_form
0,어 그거 튀겨그네,어 그거 튀겨서
1,돌멩이해 가지고 실명된 사람이 서,돌멩이해 가지고 실명된 사람 있어
2,거난 제주도는 워낙 또 돌멩이가 많아 그네,그러니까 제주도는 워낙 또 돌멩이가 많아서
3,데리고 가고 싶어 그네 할아버지가 이,데리고 가고 싶어서 할아버지가 이
4,막 영하다가 이젠,막 이렇게 하다가 이젠
...,...,...
733117,두 명이 인용 보드겡이임 이런 것도 잘도 잘 팔린댄,두 명이 인용 보드게임 이런 것도 매우 잘 팔린댄
733118,나 집에 할리갈리 이서 루미큐브도 이서 할리갈리는 보드겡이 임이 아니야,나 집에 할리갈리 있어 루미큐브도 있어 할리갈리는 보드게임이 아니야
733119,색깔 반전 시키는 것도 있댄하고 이,색깔 반전 시키는 것도 있다고 하고 이
733120,이 이가 그거 아니 쬐그만 한 거 가지고 있던데,이 이가 그거 아니야 작은 거 가지고 있던데


In [ ]:
# dialect_form 컬럼을 input_text로, standard_form 컬럼을 target_text로 변경합니다.
data.rename(columns={'dialect_form': 'input_text', 'standard_form': 'target_text'}, inplace=True)

In [ ]:
df_dialect_domain_adaptation = pd.DataFrame({
    'input_text': data['input_text'],
    'target_text': data['input_text']
})
df_dialect_domain_adaptation

,input_text,target_text
0,어 그거 튀겨그네,어 그거 튀겨그네
1,돌멩이해 가지고 실명된 사람이 서,돌멩이해 가지고 실명된 사람이 서
2,거난 제주도는 워낙 또 돌멩이가 많아 그네,거난 제주도는 워낙 또 돌멩이가 많아 그네
3,데리고 가고 싶어 그네 할아버지가 이,데리고 가고 싶어 그네 할아버지가 이
4,막 영하다가 이젠,막 영하다가 이젠
...,...,...
733117,두 명이 인용 보드겡이임 이런 것도 잘도 잘 팔린댄,두 명이 인용 보드겡이임 이런 것도 잘도 잘 팔린댄
733118,나 집에 할리갈리 이서 루미큐브도 이서 할리갈리는 보드겡이 임이 아니야,나 집에 할리갈리 이서 루미큐브도 이서 할리갈리는 보드겡이 임이 아니야
733119,색깔 반전 시키는 것도 있댄하고 이,색깔 반전 시키는 것도 있댄하고 이
733120,이 이가 그거 아니 쬐그만 한 거 가지고 있던데,이 이가 그거 아니 쬐그만 한 거 가지고 있던데


In [ ]:
train_valid_df, test_df = train_test_split(df_dialect_domain_adaptation, test_size = 0.2, random_state = 42)

In [ ]:
train_df, valid_df = train_test_split(train_valid_df, test_size = 0.25, random_state = 42)

In [ ]:
# 분할된 데이터 크기 확인
print("--- 데이터 3분할 결과 ---")
print(f"전체 데이터프레임 크기: {df_dialect_domain_adaptation.shape}")
print(f"훈련 데이터프레임 크기: {train_df.shape}")
print(f"검증 데이터프레임 크기: {valid_df.shape}")
print(f"테스트 데이터프레임 크기: {test_df.shape}")

--- 데이터 3분할 결과 ---
전체 데이터프레임 크기: (733122, 2)
훈련 데이터프레임 크기: (439872, 2)
검증 데이터프레임 크기: (146625, 2)
테스트 데이터프레임 크기: (146625, 2)


In [ ]:
# Pandas DataFrame을 Hugging Face Dataset 객체로 변환
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
# 1. 모델 및 토크나이저 불러오기
model_path = './제주도 프로젝트/standard_model_1'
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path)
model = BartForConditionalGeneration.from_pretrained(model_path)

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


In [ ]:
# 2. 데이터 토크나이징 함수 정의
def tokenize_function(examples):
    model_inputs = tokenizer(examples['input_text'], max_length=60, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['target_text'], max_length=60, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=['input_text', 'target_text'])
tokenized_valid_dataset = valid_dataset.map(tokenize_function, batched=True, remove_columns=['input_text', 'target_text'])
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=['input_text', 'target_text'])

Map:   0%|          | 0/439872 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/146625 [00:00<?, ? examples/s]

Map:   0%|          | 0/146625 [00:00<?, ? examples/s]

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.1 MB/s eta 0:00:00


In [ ]:
bleu_metric = evaluate.load('sacrebleu')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    # 모델이 생성한 예측 토큰을 문자열로 디코딩
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # -100 패딩 값 처리 후 정답 토큰을 문자열로 디코딩
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # SacreBLEU 계산을 위해 참조 문장을 리스트의 리스트 형태로 변환
    # SacreBLEU는 여러 개의 참조 문장을 지원하므로 이 형식이 필요합니다.
    decoded_labels_list = [[label] for label in decoded_labels]

    # SacreBLEU 점수 계산
    bleu_result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels_list)

    # BLEU 점수만 반환합니다.
    result = {
        "sacrebleu": bleu_result["score"]
    }

    return result

In [ ]:
# 학습 설정을 정의
training_args = Seq2SeqTrainingArguments(
    output_dir="./kobart_jeju_translation",
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=512,
    per_device_eval_batch_size=512,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=500,
    save_strategy="epoch",
    report_to='none',
    fp16=True, # ✅ 학습 시간 단축을 위한 혼합 정밀도 학습
    gradient_accumulation_steps=4, # ✅ 그래디언트 누적 단계 설정 (배치 사이즈를 2배로 사용하는 효과)
    predict_with_generate=True,
    generation_num_beams=4,
)

In [ ]:
# Seq2SeqTrainer 객체 정의
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-183854367.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
print("\n--- 제주어 -> 제주어 모델 학습 시작 (trainer) ---")
result = trainer.train()
print("\n--- 학습 완료 ---")


--- 제주어 -> 제주어 모델 학습 시작 (trainer) ---


Epoch,Training Loss,Validation Loss,Sacrebleu
1,No log,0.000053,98.773992
2,No log,0.000046,98.775392
3,0.000200,0.000038,98.777267



--- 학습 완료 ---


In [ ]:
# --- 학습된 모델 저장 ---
output_path = "./제주도 프로젝트/domain_model"
trainer.save_model(output_path)